In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

2023-08-24 11:10:17.219569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 11:10:17.953316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
import tensorflow as tf
import glob
import json
import numpy as np
import time

# import tflite_runtime.interpreter as tflite

with tf.device("CPU") as d:
    base_dir = "../data/"

    df = pd.read_csv(f"{base_dir}asl-fingerspelling/train.csv")
    # parquet_files = glob.glob('/kaggle/input/asl-fingerspelling/train_landmarks/*.parquet')

    # interpreter = tf.lite.Interpreter("/kaggle/input/asl-submission/model.tflite")
    interpreter = tf.lite.Interpreter("../models/model.tflite")

    REQUIRED_SIGNATURE = "serving_default"
    REQUIRED_OUTPUT = "outputs"

    with open (f"{base_dir}asl-fingerspelling/character_to_prediction_index.json", "r") as f:
        character_map = json.load(f)
    rev_character_map = {j:i for i,j in character_map.items()}

    found_signatures = list(interpreter.get_signature_list().keys())

    if REQUIRED_SIGNATURE not in found_signatures:
        raise KernelEvalException('Required input signature not found.')

# ../data/asl-fingerspelling/character_to_prediction_index.json
    with open (f"../models/inference_args.json", "r") as f:
        load_cols = json.load(f)
    SEL_COLS = load_cols.get("selected_columns")

    prediction_fn = interpreter.get_signature_runner("serving_default")
    tot = time.time()
    for i,(path,file_id,sequence_id,participant_id,phrase) in df.iterrows():
        data = pd.read_parquet(f'{base_dir}asl-fingerspelling/{path}', columns=SEL_COLS)
        start_time = time.time()
        output = prediction_fn(inputs=data[data.index==sequence_id])
        prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)])
        if i%10==0:
            print('infer time:', time.time()-start_time, )
            print('pred:', prediction_str)
            print('gt:',phrase)
            print(' ')
        if i==100:
            break
    print('avg time',(time.time()-tot)/i)


ModuleNotFoundError: No module named 'tflite_runtime'